In [ ]:
from main import ModelSpecLarge, init_training_data
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
batchsize = 20

network = ModelSpecLarge()
_, it, valset = init_training_data("../data/train.csv", batchsize=batchsize, valsetsize=100, epochs=50)
network.init_training(batchsize=batchsize, validation_set=valset)

In [ ]:
for batch in it:
    (training_loss, training_accuracy, validation_loss, validation_accuracy) = network.train_minibatch(batch)
    print(f"{validation_loss:6.3f}, {validation_accuracy:4.2f}\r", end="")


In [ ]:
network.cleanup_training()

In [ ]:
network.save_params()

Testing

In [ ]:
def file_it(path, skiplines=0):
    with open(path) as f:
        for _ in range(skiplines):
            f.readline()
        yield from f

it = map(
    lambda line: np.fromstring(line, sep=",", dtype=np.uint8),
    file_it("../data/test.csv", skiplines=1),
)

In [ ]:
data = next(it)
n = network.feedforward(data/255)
print(n)
fig,ax=plt.subplots()
ax.imshow(data.reshape((28,28)), cmap="gray", vmin = 0, vmax = 255)
ax.tick_params(axis='both', which='major', labelsize=14, length=10, width=2)
plt.show()